In [ ]:
import asyncio
from math import log, sin, cos, tan, exp, sqrt, pi
from time import time, sleep
from random import randrange
import torch
import numpy as np
from testbed import UTF8Dataset, MLPLM, TransformerLM, AdamW, Learner

## Scheduling helpers

In [ ]:
constant = lambda c: lambda step: c
linear_warmup_then_decay = lambda lr, warmup: lambda n: lr*(n/warmup) if n < warmup else lr*(warmup/n)

## MLP Language Model

In [ ]:
model = MLPLM(n_vocab_in=256, n_vocab_out=256, n_ctx=64,
              d_model=64, d_hidden=8192, nonlinearity="GELU").to('cuda')

optimizer = AdamW(parameters=model.parameters(), eps=constant(1e-4), 
                  lr=linear_warmup_then_decay(lr=5e-5,warmup=256), 
                  beta1=constant(0.9), beta2=constant(0.999),
                  weight_decay=constant(0.01), initial_step=0)

dataset = UTF8Dataset()

config = {"model": model, "optimizer": optimizer, "dataset": dataset}

In [ ]:
learner = Learner(config=config)

In [ ]:
reader, writer = await asyncio.open_connection('127.0.0.1', 8888)
async def train(n_steps, batch_size, example_length, stream):
    try:
        for _ in range(n_steps):
            await asyncio.sleep(.01)
            loss = np.sum(learner.step(batch_size, example_length))/batch_size
            stream.write(bytes(struct.pack("f", value)))
            await stream.drain()
    except Exception as e:
        return e
    return None

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()
import time, math
from threading import Thread, Event, Lock

class Ticker:
    def __init__(self, x='time', y='mean_loss', stream):
        self.stream = stream
        self.data = []
        self.x = x
        self.y = y
        self.bokeh = {}
        self.task = None

    def __repr__(self):
        self.bokeh["figure"] = figure(x_axis_label=self.x.replace('_',' '),
                                      y_axis_label=self.y.replace('_',' '),
                                      tools="pan,wheel_zoom,box_zoom,reset")
        self.bokeh["figure"].axis.major_label_text_font_size = "24px"
        self.bokeh["data"] = self.bokeh["figure"].line([],[])
        self.bokeh_handle = show(self.bokeh["figure"], notebook_handle=True)
        if self.task is None:
            self.task = asyncio.create_task(self._update_loop(self.x, self.y, self.bokeh,
                                                              self.bokeh_handle, self.stream)
        return ""

    def __del__(self):
        self.task.cancel()

    @staticmethod
    async def _update_loop(x, y, bokeh, bokeh_handle, reader):
        tick = 0
        try:
            await asyncio.sleep(1.0)
            data = {x: [item[x] for item in trainer.metrics[tick:]], y: [item[y] for item in trainer.metrics[tick:]]}
            if len(data) > 0:
                if stop_event.is_set():
                    break
                bokeh["data"].data_source.stream({'x': data[x], 'y': data[y]})
                tick = len(trainer.metrics)
            push_notebook(handle=bokeh_handle)
        except Exception as e:
            print(e)
            break


In [ ]:
t = asyncio.create_task(train(2**20, 256, 65, 'log.txt'))

In [ ]:
t.done()

In [ ]:
!tail -n 1 log.txt

## Transformer Language Model

In [ ]:
model = TransformerLM(n_vocab_in=256, n_vocab_out=256, max_ctx=128, d_model=256,
                      d_k=16, d_v=16, n_heads=16, d_hidden=256, n_layers=8, p_dropout_embedding=0.0,
                      p_dropout_attn_mat=0.0, p_dropout_attn_out=0.0, p_dropout_mlp=0.0).to('cuda')

optimizer = AdamW(parameters=model.parameters(), eps=constant(1e-4),
                  lr=linear_warmup_then_decay(lr=1e-4,warmup=10000), 
                  beta1=constant(0.9), beta2=constant(0.999), weight_decay=constant(0.01),
                  initial_step=0)

dataset = UTF8Dataset()

config = {"model": model, "optimizer": optimizer, "dataset": dataset}

In [ ]:
learner = Learner(config=config)

In [ ]:
learner.step(256,33)

## Plotting

In [ ]:
ticker = StatsTicker(trainer,  x='step', y='mean_loss')
ticker

In [ ]:
trainer.update("optimizer", lr=1e-03)

In [ ]:
trainer.update("dataset", batch_size=8192)

In [ ]:
result = ""


In [ ]:
more = ''.join(list(trainer.autocomplete(result[:128],n_generate=256, max_ctx=128)))
print(more)

In [ ]:
with open('gibberish.txt', 'w') as outfile:
    outfile.write(result)

In [ ]:
async def foo():
    global result
    more = ''.join(list(trainer.autocomplete(n_generate=256, max_ctx=128)))
    result += more
    with open('gibberish.txt', 'a') as outfile:
        outfile.write(more)
    return result

In [ ]:
for _ in range(2400):
    sleep(15)
    t = asyncio.create_task(foo())
    await t

In [ ]:
trainer.metrics[-1]

In [ ]:
trainer.save("checkpoint.pt")

In [ ]:
trainer.load('checkpoint.pt')

### SmoothPlot

In [ ]:
import scipy.ndimage

def smoother(X, Y, lag):
    Y = np.cumsum(Y)
    return X[lag:], (Y[lag:] - Y[:-lag])/lag

def gsmoother(X, Y, lag):
    X = X[lag:-lag]
    Y = scipy.ndimage.gaussian_filter1d(Y, sigma=lag)[lag:-lag]
    return (X, Y)

class SmoothPlot(LinePlot):
    def __init__(self, trainer, lag=100, log=None):
        L = np.array([[x['step'],x['mean_loss']] for x in trainer.metrics])
        n = len(L[:,0])
        k = n//1000 + 1
        X = L[:,0]
        Y = L[:,1]
        X,Y = gsmoother(X, Y, lag)
        X = X[::k]
        Y = Y[::k]
        if log:
            X = np.log(X)/math.log(2)
        super().__init__(X, Y)

class GaussianSmoothedLossRate(LinePlot):
    def __init__(self, trainer, lag=100, log=None):
        L = np.array([[x['step'],x['mean_loss']] for x in trainer.metrics])
        X = L[1:,0]
        Y = -L[1:,1] + L[:-1,1]
        X,Y = gsmoother(X, Y, lag)
        if log:
            X = np.log(X)/math.log(2)
        super().__init__(X, Y)

In [ ]:
SmoothPlot(trainer, lag=10, log=False)

In [ ]:
GaussianSmoothedLossRate(trainer, lag=10000, log=False)